<a href="https://colab.research.google.com/github/SubhashishM/GENAI/blob/main/Assignment_Encoding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Import Twitter Data


In [ ]:
!pip install -q kaggle

In [ ]:
from google.colab import files
files.upload()

In [ ]:
!mkdir ~/.kaggle

In [ ]:
!cp kaggle.json ~/.kaggle/

In [ ]:
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d thoughtvector/customer-support-on-twitter

In [ ]:
!unzip /content/customer-support-on-twitter.zip

In [ ]:
import pandas as pd
#df = pd.read_csv('/content/twcs/twcs.csv')
df = pd.read_csv('/content/sample.csv')
df

#Data Extraction

In [ ]:
import pandas as pd
data = ' '.join(str(item) for item in df['text'])

In [ ]:
import textblob
from textblob import TextBlob
txtblob=TextBlob(data)
txtblob.correct().string

In [ ]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

def remove_stopwords(text):
  new_text=[]
  for word in text.split():
    if word not in stopwords.words("english"):
      new_text.append(word)
  return " ".join(new_text)

remove_stopwords(data)

In [ ]:
import re
def remove_url(text):
    pattern=re.compile(r'https?://\S+|www\.\S+')
    return pattern.sub("",text)

data_without_url = remove_url(data)
data_without_url

In [ ]:
def remove_at(text):
  new_text=[]
  for word in text.split():
    if "@" not in word:
      new_text.append(word)
  return " ".join(new_text)

data_without_url_at = remove_at(data_without_url)
data_without_url_at

In [ ]:
import string

def remove_punc(text):
    for char in string.punctuation:
        text=text.replace(char,"")
    return text

data_without_url_at_punc = remove_punc(data_without_url_at)
data_without_url_at_punc

In [ ]:
!pip install emoji
import emoji
import re
def remove_emojis_manually(text):
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F700-\U0001F77F"  # alchemical symbols
                               u"\U0001F780-\U0001F7FF"  # Geometric Shapes Extended
                               u"\U0001F800-\U0001F8FF"  # Supplemental Arrows-C
                               u"\U0001F900-\U0001F9FF"  # Supplemental Symbols and Pictographs
                               u"\U0001FA00-\U0001FA6F"  # Chess Symbols
                               u"\U0001FA70-\U0001FAFF"  # Symbols and Pictographs Extended-A
                               u"\U00002702-\U000027B0"  # Dingbats
                               u"\U000024C2-\U0001F251"
                               "]+", flags=re.UNICODE)
    clean_text = emoji_pattern.sub(r'', text)
    return clean_text

data_without_url_at_punc_emoji = remove_emojis_manually(data_without_url_at_punc)
data_without_url_at_punc_emoji

In [ ]:
import nltk
from nltk.tokenize import word_tokenize,sent_tokenize
nltk.download("punkt")
word_tokenize(data_without_url_at_punc_emoji)

In [ ]:
sent_tokenize(data_without_url_at_punc_emoji)

# stemming and lemmetization

In [ ]:
from nltk.stem import PorterStemmer

def stemming(text):
    obj=PorterStemmer()

    stem_word=[obj.stem(word) for word in text.split()]

    return stem_word

In [ ]:
stemming(data_without_url_at_punc_emoji)

In [ ]:
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
def lammatization(text):
    words=text.split()

    lemmetizer=WordNetLemmatizer()

    lemetized_word=[lemmetizer.lemmatize(word) for word in words]

    return lemetized_word

In [ ]:
lammatization(data_without_url_at_punc_emoji)

# Encoding

In [ ]:
df['text']

In [ ]:
from sklearn.preprocessing import OneHotEncoder

# Instantiate the OneHotEncoder
encoder = OneHotEncoder(max_categories=10, sparse=False)

# Fit and transform the categorical column
one_hot_encoded = encoder.fit_transform(df[['text']])

# Display the result
pd.DataFrame(one_hot_encoded, columns=encoder.get_feature_names_out(['text']))

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
BOW=CountVectorizer()
document_matrix=BOW.fit_transform(df["text"])

In [ ]:
document_matrix[6].toarray()

In [ ]:
BOW.vocabulary_

In [ ]:
bigram=CountVectorizer(ngram_range=(2,2))
bigramvocab=bigram.fit_transform(df["text"])
bigram.vocabulary_

In [ ]:
bigram=CountVectorizer(ngram_range=(3,3))
bigramvocab=bigram.fit_transform(df["text"])
bigram.vocabulary_

In [ ]:
bigram=CountVectorizer(ngram_range=(1,3))
bigramvocab=bigram.fit_transform(df["text"])
bigram.vocabulary_

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf=TfidfVectorizer()
tfidf.fit_transform(df["text"]).toarray()

In [ ]:
tfidf.get_feature_names_out()

In [ ]:
tfidf.idf_

#OHE vs BOW vs N-grams vs TF-IDF

***One-Hot Encoding (OHE):***

**Advantages:**

1. Simple Representation: OHE is a straightforward and easy-to-understand representation, where each category is represented by a binary indicator.
2. No Assumption of Relationships: OHE doesn't assume any relationships between categories. Each category is treated independently.
3. Useful for Categorical Data: OHE is particularly useful for dealing with categorical data when there is no ordinal relationship between categories.
4. Interpretability: The resulting representation is interpretable, making it easy to understand the contribution of each category to the model.
5. No Parameter Tuning: OHE does not involve tuning parameters, making it easy to implement and use in various models.
**Disadvantages**
1. High Dimensionality: OHE can lead to a high-dimensional representation, especially when dealing with a large number of categories, which can be computationally expensive.
2. Sparse Representation: The resulting matrix is often sparse, containing many zeros, which can be inefficient for storage and computation.
3. Loss of Order Information: OHE does not capture any ordinal relationship between categories. All categories are treated as equal.
4. May Not Handle Large Vocabularies Well: When dealing with large vocabularies, OHE can become impractical due to the resulting high-dimensional space.
5. Collinearity Issues: OHE can introduce multicollinearity when used in linear models if one category is a perfect predictor of another.
---
***Bag-of-Words (BOW):***

**Advantages:**
1. Simple and Intuitive: BOW is a simple and intuitive representation that captures the frequency of words in a document.
2. Versatile: BOW can be used for various natural language processing (NLP) tasks, such as text classification, sentiment analysis, and information retrieval.
3. Easy to Implement: Implementing BOW is straightforward and can be done with minimal preprocessing.
4. Robust to Noise: BOW can be robust to noise in the data, as it focuses on the occurrence of words rather than their order.
5. Scalability: BOW can handle large datasets efficiently and is scalable.
**Disadvantages:**
1. Loss of Sequence Information: BOW disregards the order and sequence of words, which may result in a loss of important context.
2. No Semantics: BOW does not capture semantic relationships between words.
3. Fixed-Length Representation: BOW represents documents as fixed-length vectors, which may not be suitable for variable-length documents.
4. Sparse Representation: Similar to OHE, BOW can lead to a sparse representation with many zero entries.
5. Ignores Word Importance: BOW treats all words equally, ignoring variations in importance.
---
***N-grams:***
**Advantages:**
1. Captures Local Context: N-grams capture local context by considering sequences of adjacent words.
2. Handles Variable-Length Context: N-grams allow for a variable-length context, capturing different levels of context based on the chosen N.
3. Retains Some Order Information: While not capturing full syntactic structure, N-grams retain some order information by considering adjacent word pairs.
4. Applicable to Multiple Tasks: N-grams can be applied to various NLP tasks, including language modeling, sentiment analysis, and named entity recognition.
5. Combines Simplicity with Improved Context: Compared to BOW, N-grams provide improved context information while maintaining simplicity.
**Disadvantages:**
1. Exponential Growth in Vocabulary Size: The vocabulary size can grow exponentially with the order of N, leading to increased computational complexity.
2. Sparse Representation: Like BOW, N-grams can result in a sparse representation, especially with large vocabularies.
3. May Capture Noise: Longer N-grams may capture noise or spurious patterns that do not contribute to the underlying meaning.
4. Limited Understanding of Global Context: N-grams have limitations in understanding global context or long-range dependencies in text.
5. Dependent on Corpus Size: The effectiveness of N-grams can depend on the size of the training corpus.

---
***TF-IDF (Term Frequency-Inverse Document Frequency):***

**Advantages:**
1. Weighting for Importance: TF-IDF assigns weights to words based on their importance in a document and across the corpus.
2. Handles Common Words: TF-IDF mitigates the issue of common words by down-weighting terms that appear frequently across documents.
3. Promotes Discriminative Features: TF-IDF emphasizes terms that are discriminative for a specific document.
4. Reduces Noise: By down-weighting common terms, TF-IDF reduces the impact of noise in the data.
5. Applicable to Various NLP Tasks: TF-IDF is versatile and can be used in tasks such as text classification, information retrieval, and clustering.
**Disadvantages:**
1. Ignores Word Order: TF-IDF, like BOW, ignores word order and does not capture syntactic or semantic relationships.
2. Sparse Representation: The resulting TF-IDF matrix can be sparse, especially with a large vocabulary.
3. Doesn't Consider Document Length: TF-IDF does not explicitly consider the length of documents, which may be important in certain contexts.
4. Sensitive to Tokenization: The effectiveness of TF-IDF can be sensitive to the chosen tokenization method.
5. May Require Tuning: Tuning parameters, such as the choice of stop words and the smoothness factor for IDF, may be required for optimal performance.